In [ ]:
from utils.data_manipulation import OSMHandler
import os
from geopy.geocoders import Nominatim

In [ ]:
# SZUKANIE PO ADRESIE:
geolocator = Nominatim(user_agent = "my_geocoder")

address = ('Żurawica')
location = geolocator.geocode(address)

if location:
    print("Współrzędne dla adresu '{}':".format(address))
    print("Szerokość geograficzna:", location.latitude)
    print("Długość geograficzna:", location.longitude)
    lat = location.latitude
    lon = location.longitude
else:
    print("Nie można znaleźć współrzędnych dla adresu '{}'. Spróbuj podać inny adres.".format(address))
    lat = 50.0469432
    lon = 19.997153435836697

In [ ]:
dic = {'amenity': ['bar', 'biergarten', 'cafe', 'fast_food', 'food_court', 'ice_cream', 'pub', 'restarurant', 'school', 'library', 'atm', 'bank', 'hospital', 'veterinary', 'pharmacy',
                  'clinic', 'cinema', 'nightclub', 'money_transfer', 'payment_terminal'],
       'building': ['church'],
        'leisure': ['fintess_centre']}

In [ ]:
path = os.path.join(os.getcwd(), 'maps', 'malopolskie.pbf')
print(path)

#Obszar w km w około którego szukamy:
radius = 3

h = OSMHandler(lat, lon, radius, dic)
h.apply_file(path)

Dataframe = h.acc()

#Ustawienie datatype
Dataframe['category'] = Dataframe['category'].astype("string")
Dataframe['subcategory'] = Dataframe['subcategory'].astype("string")
Dataframe['name'] = Dataframe['name'].astype("string")

In [ ]:
def get_address(lat, lon):
    location = geolocator.reverse((lat, lon))
    return location.address if location else None

Dataframe['address'] = Dataframe.apply(lambda row: get_address(row['lat'], row['lon']), axis=1)
Dataframe['address'] = Dataframe['address'].astype("string")

In [ ]:
print(Dataframe.info())

In [ ]:
# Dataframes segmentation:
import pandas as pd

food = ['fast_food', 'food_court', 'restaurant']
education = ['school', 'library']
financial = ['bank', 'atm', 'money_transfer', 'payment_terminal']
hospitals = ['clinic', 'hospital', 'pharmacy']

food_dataframe = Dataframe[Dataframe['subcategory'].isin(food)].copy()
education_dataframe = Dataframe[Dataframe['subcategory'].isin(education)].copy()
financial_dataframe = Dataframe[Dataframe['subcategory'].isin(financial)].copy()
hospitals_dataframe = Dataframe[Dataframe['subcategory'].isin(hospitals)].copy()

In [ ]:
#VISUALIZATION OF DATA:
import folium


vis_dataframe = food_dataframe

map = folium.Map(location = (lat, lon), zoom_start = 14)
folium.Marker(location = (lat, lon), popup ='INIT', icon=folium.Icon(color='red')).add_to(map)
folium.Circle(location = (lat, lon), color = 'blue',radius = radius*1000,fill_color='lightblue', fill_opacity=0.2).add_to(map)

for index, row in vis_dataframe.iterrows():
    lat_f = row['lat']
    lon_f = row['lon']
    name = row['name']

    folium.Marker(location = (lat_f, lon_f), popup=f'{name}').add_to(map)
    

map

In [ ]:
# SZUKANIE ADRESU PO LOKALIZACJI:

# lat = 49.83611855
# lon = 22.76256295511138

address = geolocator.reverse((lat, lon))

if address:
    print(type(address.address))
else:
    print('Nie znaleziono')